In [ ]:
from cassandra.cluster import Cluster

cluster = Cluster(["61.80.148.154"], port=30003)
session = cluster.connect()

In [ ]:
# Execution Profiles

from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory
from cassandra.query import ConsistencyLevel

profile = ExecutionProfile(
    load_balancing_policy=WhiteListRoundRobinPolicy(["61.80.148.154"]),
    retry_policy=DowngradingConsistencyRetryPolicy(),
    consistency_level=ConsistencyLevel.LOCAL_QUORUM,
    serial_consistency_level=ConsistencyLevel.LOCAL_SERIAL,
    request_timeout=15,
    row_factory=tuple_factory
)

cluster = Cluster(["61.80.148.154"], port=30003, execution_profiles={EXEC_PROFILE_DEFAULT : profile})
session = cluster.connect()

In [ ]:
# Prepared Statements
# Queries that are parsed by Cassandra and then saved for later use. When the driver uses a prepared statement,
# it only needs to send the values of parameters to bind. This lowers network traffic and CPU utilization
# within Cassandra because Cassandra does not have to re-parse the query each time.

user_lookup_stmt = session.prepare("SELECT * FROM users WHERE user_id=?")

users = []
for user_id in user_ids_to_query:
    user = session.execute(user_lookup_stmt, [user_id])
    users.append(user)

In [ ]:
import pickle

with open("user_similarity.pkl", "rb") as f:
    data = pickle.load(f)

In [ ]:
# Execution Profiles

from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory
from cassandra.query import ConsistencyLevel

profile = ExecutionProfile(
    load_balancing_policy=WhiteListRoundRobinPolicy(["61.80.148.154"]),
    retry_policy=DowngradingConsistencyRetryPolicy(),
    consistency_level=ConsistencyLevel.LOCAL_QUORUM,
    serial_consistency_level=ConsistencyLevel.LOCAL_SERIAL,
    request_timeout=15,
    row_factory=tuple_factory
)

cluster = Cluster(["61.80.148.154"], port=30003, execution_profiles={EXEC_PROFILE_DEFAULT : profile})
session = cluster.connect()

In [ ]:
# INSERT image similarity data into cassandra
for user in data.keys():
    user_similarity_stmt = session.prepare(
    f"INSERT INTO user_profile_test.similarity_test (user_id, similarity, target_id) VALUES ({int(user)}, ?, ?)"
    )
    for user_similarity, target_id in data[user]:
        session.execute(user_similarity_stmt, [user_similarity, int(target_id)])

In [ ]:
# Create user_rating data 
import random
from collections import defaultdict

user_rating = defaultdict(list)

for i in range(600):
    user_id = str(random.randrange(0, 1001))
    while user_id in user_rating:
        user_id = str(random.randrange(0, 1001))

    for i in range(random.randrange(1, 20)):
        target_id = str(random.randrange(0, 1001))
        while target_id in user_rating[user_id]:
            target_id = str(random.randrange(0, 1001))
        
        random_rating = random.randrange(1, 11)
        user_rating[user_id].append([target_id, random_rating])

import pickle
with open("user_rating.pkl", "wb") as f:
    pickle.dump(user_rating, f)

In [ ]:
# INSERT user_rating datat into cassandra
for user_id in user_rating.keys():
    user_rating_stmt = session.prepare(
        f"INSERT INTO user_profile_test.rating_test (user_id, rating, target_id) VALUES ({int(user_id)}, ?, ?)"
    )
    for target_id, rating in user_rating[user_id]:
        session.execute(user_rating_stmt, [-rating, int(target_id)])

In [ ]:
# Load user_rating data from cassandra
rating_query = session.prepare("SELECT * FROM user_profile_test.rating_test WHERE user_id=?")

user_id = 16
user_favored_list = session.execute(rating_query, [int(user_id)])
for user_id, rating, target_id in user_favored_list:
    print(user_id, rating, target_id)

In [ ]:
rating_query = session.prepare("SELECT * FROM user_profile_test.rating_test WHERE user_id=?")

user_id = 16
user_favored_list = [target_id for _, _, target_id in session.execute(rating_query, [int(user_id)])]

similarity_query = "SELECT * FROM user_profile_test.similarity_test WHERE user_id=%s"
similarity_query_futures = [session.execute_async(similarity_query, [target_id]) for target_id in user_favored_list[:min(len(user_favored_list), 5)]]

# wait for them to complete and use the results
for future in similarity_query_futures:
    rows = future.result()
    for row in rows:
        print(row)
    print("@@@@@@@@@@")

In [ ]:
# Non-prepared statements
session.execute(
    """
    INSERT INTO users (name, credits, user_id)
    VALUES (%s, %s, %s)
    """,
    ("John O'Reilly", 42, uuid.uuid1()),
)

session.execute(
    """
    INSERT INTO users (name, credits, user_id, username)
    VALUES (%(name)s, %(credits)s, %(user_id)s, %(name)s)
    """,
    {'name': "John O'Reilly", 'credits': 42, 'user_id': uuid.uuid1()}
)

In [ ]:
# Execute_async()
# Instead of waiting for the query to complete and returning rows directly, this method almost immediately
# returns a ResponseFuture object. There are two ways of getting the final result from this object.

# The first, by calling result()
from cassandra import ReadTimeout

query = "SELECT * FROM users WHERE user_id=%s"
future = session.execute_async(query, [user_id])

# ... do some other work
try:
    rows = future.result()
    user = rows[0]
    print(user.name, user.age)
except ReadTimeout:
    log.exception("Query timed out")

In [ ]:
# build a list of futures
futures = []
query = "SELECT * FROM users WHERE user_id=%s"
for user_id in ids_to_fetch:
    futures.append(session.execute_async(query, [user_id]))

# wait for them to complete and use the results
for future in futures:
    rows = future.result()
    print(rows[0].name)

In [ ]:
# Instead of calling result(), you can attach callback and errback functions through the
# add_callback(), add_errback(), add_callbacks() methods.
def handle_success(rows):
    user = rows[0]
    try:
        process_user(user.name, user.age, user.id)
    except Exception:
        log.error("Failed to process user %s", user.id)
        # don't re-raise errors in the callback

def handle_error(exception):
    log.error("Failed to fetch user info: %s", exception)

future = session.execute_async(query)
future.add_callbacks(handle_success, handle_error)

In [ ]:
# Setting a Consistency Level
# The consistency level used for a query determines how many of the replicas of the data you are
# interacting with need to respond for the query to be considered a success.
# You can specify a different default by setting the ExecutionProfile.consistency_level 
# for the execution profile with key EXEC_PROFILE_DEFAULT. 
# To specify a different consistency level per request, wrap queries in a SimpleStatement

from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement

query = SimpleStatement(
    "INSERT INTO users (name, age) VALUES (%s, %s)",
    consistency_level = ConsistencyLevel.QUORUM,
)
session.execute(query, ("John", 42))

In [ ]:
# Execution Profiles

from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory
from cassandra.query import ConsistencyLevel

profile = ExecutionProfile(
    load_balancing_policy=WhiteListRoundRobinPolicy(["61.80.148.154"]),
    retry_policy=DowngradingConsistencyRetryPolicy(),
    consistency_level=ConsistencyLevel.LOCAL_QUORUM,
    serial_consistency_level=ConsistencyLevel.LOCAL_SERIAL,
    request_timeout=15,
    row_factory=tuple_factory
)

cluster = Cluster(['cassandra.us-east-2.amazonaws.com'], port=30003, execution_profiles={EXEC_PROFILE_DEFAULT : profile})
session = cluster.connect()

In [26]:
! pip install s3fs

In [25]:
! pip install boto3==1.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: boto3
    Found existing installation: boto3 1.27.0
    Uninstalling boto3-1.27.0:
      Successfully uninstalled boto3-1.27.0


In [15]:
from cassandra.cluster import Cluster
from ssl import SSLContext, PROTOCOL_TLSv1_2 , CERT_REQUIRED
from cassandra.auth import PlainTextAuthProvider
import boto3

ssl_context = SSLContext(PROTOCOL_TLSv1_2)
ssl_context.load_verify_locations('/Users/jaeho/.ssh/sf-class2-root.crt')
ssl_context.verify_mode = CERT_REQUIRED

auth_provider = PlainTextAuthProvider(
    username="keyspace_user+1-at-895636194780", 
    password="j/vlAkLrtf3zlGqu5DFWgmr0/2IvNJurhliLJ1w2bDQ=",
    )

# Use this instead of the above line if you want to use the Default Credentials and not bother with a session.
# auth_provider = SigV4AuthProvider()

cluster = Cluster(
    ['cassandra.ap-northeast-2.amazonaws.com'], 
    ssl_context=ssl_context,
    auth_provider=auth_provider,
    port=9142,
    )
session = cluster.connect()
rows = session.execute('select * from member.info')
for row in rows:
    print(row.photo)

/var/folders/23/z66xfw69593_wvynw4k7bb080000gn/T/ipykernel_49824/2401933602.py:6: DeprecationWarning: ssl.PROTOCOL_TLSv1_2 is deprecated
  ssl_context = SSLContext(PROTOCOL_TLSv1_2)


None
{1: '사진 경로'}
{1: '사진 경로'}
{1: 'C:\\Users\\gjaischool\\Music\\ws\\09.nov\\src\\main\\webapp\\\\1.jpg'}
None
{1: 's3://simkoong-s3/simkoong.jpg', 2: 's3://simkoong-s3/simkoong.jpg', 3: 's3://simkoong-s3/simkoong.jpg', 4: 's3://simkoong-s3/simkoong.jpg'}
None
None
{1: 's3://simkoong-s3/iu.png', 2: 's3://simkoong-s3/iu3.jpg', 3: 's3://simkoong-s3/iu2.jpg', 4: 'C:/Users/gjaischool/Music/ws/새 폴더/FinalProject_web/src/main/webapp/302.jpg'}
None
{1: '사진 경로'}
None
None
None
None
None
None
None
{1: 'default.png', 2: 'default.png', 3: 'default.png', 4: 'default.png'}
{1: '사진 경로'}
{2: 'C:/Users/gjaischool/Music/ws/새 폴더/FinalProject_web/src/main/webapp/6.jpg'}
None
None
{1: '사진 경로'}
{1: '사진 경로'}
None
None
{1: 'C:/Users/gjaischool/Music/ws/09.nov/src/main/webapp/2.jpg'}
None
None
{1: 'default.png', 2: 'default.png', 3: 'default.png', 4: 'default.png'}
{1: '사진 경로'}
{1: 'C:/Users/gjaischool/Music/ws/09.nov/src/main/webapp//2.jpg'}
{1: '사진 경로'}
None
None
{1: 'default.png', 2: 'default.png', 3: 'def